In [1]:
# Perform my imports

import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import heapq
import json
import time
import random
import os
import re
import math
import pandas as pd
import sys
import numpy as np
from typing import Optional, List
from transformers import AutoTokenizer, LlamaForCausalLM, BitsAndBytesConfig
import json, csv
from sklearn.model_selection import train_test_split

sys.path.append("..")
from CeRTS_beam_multi import *
from CeRTS_utils import *
sys.path.remove("..")

from generation_functions import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Initializations

In [2]:
df_val = pd.read_csv("clinical_data/validation.csv")

json_template = {
    "asthma": "yes|no",
    "smoking": "yes|no",
    "pneu": "yes|no",            
    "common_cold": "yes|no",
    "pain": "yes|no",
    "fever": "high|low|no",
    "antibiotics": "yes|no"
}

features = json_template.keys()

# Now mess around with my data to just get it split up into sections

### Load in Model

In [3]:
model_path = "models/llama-3.2-3b-instruct"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# load model (using float16 + device map for GPU efficiency)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto" 
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Now Test the Model

In [6]:
n = 52

# Model
df_val.loc[n, 'advanced_text']
prompt = gen_prompt_no_shot(df_val.loc[n, 'advanced_text'], features, json_template)
print(prompt)

[{'role': 'system', 'content': 'You are an information extraction function.\nReturn EXACTLY one JSON object and nothing else (no prose, no code fences).\nAll boolean-like fields MUST be STRING LITERALS \'yes\' or \'no\' (never 1/0, never true/false).\nUse lowercase literals exactly as shown. Do not add, remove, or rename keys.\nSchema (types and enums are STRICT):\n{"type":"object","additionalProperties":false,"properties":{"asthma":{"type":"string","enum":["yes","no"]},"smoking":{"type":"string","enum":["yes","no"]},"pneu":{"type":"string","enum":["yes","no"]},"common_cold":{"type":"string","enum":["yes","no"]},"pain":{"type":"string","enum":["yes","no"]},"fever":{"type":"string","enum":["high","low","no"]},"antibiotics":{"type":"string","enum":["yes","no"]},}}'}, {'role': 'user', 'content': 'Now extract the following fields: asthma, smoking, pneu, common_cold, pain, fever, antibiotics.\nOutput ONLY a single JSON object that matches this exact schema: {"asthma": "yes|no", "smoking": "

In [7]:
# convert messages into a proper model input
prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)

# tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

# decode to string
generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

# 
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'{"asthma": "no", "smoking": "no", "pneu": "no", "common_cold": "no", "pain": "yes", "fever": "no", "antibiotics": "no"}'